## **Install dependencies** 🛫

In [11]:
%%capture

# Install and update essential tools for managing Python packages
get_ipython().system('python -m pip install --upgrade pip setuptools wheel')

# Install libraries for downloading and handling torrent files
get_ipython().system('python -m pip install lbry-libtorrent wget torf')

# Install system-level dependency for torrent functionality
get_ipython().system('apt install python3-libtorrent -y')

## **Download torrent** ⏬

In [10]:
import zipfile
import os
import libtorrent as lt
import time
import datetime
import shutil

# --- Torrent Download Settings ---
DOWNLOAD_PATH = '/content/Torrent/'
STORAGE_MODE = lt.storage_mode_t(2)

params = {
    'save_path': DOWNLOAD_PATH,
    'storage_mode': STORAGE_MODE,
}

# --- Start Libtorrent Session ---
ses = lt.session()
ses.listen_on(6881, 6891)

# --- Get Torrent/Magnet Link from User ---
link = input("Enter Torrent file link or Magnet link: ")

# --- Handle Torrent File Conversion ---
if link.endswith('.torrent'):
    import wget
    from torf import Torrent

    TORRENT_FILE = 'torrent.torrent'

    if os.path.exists(TORRENT_FILE):
        os.remove(TORRENT_FILE)

    wget.download(link, TORRENT_FILE)
    torrent_info = Torrent.read(TORRENT_FILE)
    link = str(torrent_info.magnet(name=True, size=False, trackers=False, tracker=False))

# --- Start Download ---
print("Using link:", link)
handle = lt.add_magnet_uri(ses, link, params)
handle.set_sequential_download(0)  # Download in parallel
ses.start_dht()

begin = time.time()
print(datetime.datetime.now())
print("Downloading Metadata...")

while not handle.has_metadata():
    time.sleep(1)

print("Got Metadata, Starting Torrent Download...")
print("Starting:", handle.name())

# --- Monitor and Display Progress ---
while handle.status().state != lt.torrent_status.seeding:
    status = handle.status()
    state_str = ['queued', 'checking', 'downloading metadata',
                 'downloading', 'finished', 'seeding', 'allocating']

    print(
        '%.2f%% complete (down: %.1f kb/s up: %.1f kB/s peers: %d) %s ' %
        (status.progress * 100, status.download_rate / 1000,
         status.upload_rate / 1000, status.num_peers, state_str[status.state])
    )
    time.sleep(5)

# --- Download Complete ---
end = time.time()
print(handle.name(), "COMPLETE")
print("Elapsed Time: ", int((end - begin) // 60), "min :", int((end - begin) % 60), "sec")
print(datetime.datetime.now())

Input Torrent file link or Magnet link and Press Enter: magnet:?xt=urn:btih:95821687520e75998a15237cf9e0073ffbd0af2a&dn=%E5%B0%8F%E9%A9%AC%E5%A4%A7%E8%BD%A6
magnet:?xt=urn:btih:95821687520e75998a15237cf9e0073ffbd0af2a&dn=%E5%B0%8F%E9%A9%AC%E5%A4%A7%E8%BD%A6
2025-02-25 08:35:00.954060
Got Metadata, Starting Torrent Download...
Starting 小马大车
0.01% complete (down: 42.7 kb/s up: 1.7 kB/s peers: 1) downloading 
0.02% complete (down: 206.7 kb/s up: 6.6 kB/s peers: 2) downloading 
0.08% complete (down: 600.0 kb/s up: 18.3 kB/s peers: 2) downloading 
0.14% complete (down: 827.1 kb/s up: 26.2 kB/s peers: 2) downloading 
0.20% complete (down: 918.2 kb/s up: 27.1 kB/s peers: 1) downloading 
0.70% complete (down: 4987.1 kb/s up: 151.2 kB/s peers: 1) downloading 
0.73% complete (down: 2085.9 kb/s up: 63.2 kB/s peers: 0) downloading 
0.73% complete (down: 683.5 kb/s up: 20.7 kB/s peers: 0) downloading 
0.73% complete (down: 224.0 kb/s up: 6.8 kB/s peers: 0) downloading 
0.73% complete (down: 73.4 kb

KeyboardInterrupt: 

## **Zip the files** 📐

In [12]:
import zipfile
import os

def zip_downloaded_torrent(torrent_directory='content/Torrent/', zip_filename='Downloaded-Torrent.zip'):
    """
    Zips the contents of the downloaded torrent directory into a single ZIP file.

    Args:
        torrent_directory: The path to the directory containing the downloaded torrent files.
        zip_filename: The name of the ZIP file to be created.
    """
    print('Zipping started...')

    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, _, files in os.walk(torrent_directory):
            for file in files:
                # Construct the full file path
                file_path = os.path.join(root, file)

                # Calculate the relative path for archiving
                archive_path = os.path.relpath(file_path, os.path.join(torrent_directory, '..'))

                # Add the file to the ZIP archive
                zip_file.write(file_path, archive_path)

    print('\nALL DONE!')

# Call the function to zip the downloaded torrent
zip_downloaded_torrent()

Zipping started...

ALL DONE!
